In [ ]:
import sqlite3
import json
from neo4j import GraphDatabase
import sys
import os
import pandas as pd
import datetime as datetime
from datetime import timedelta


In [ ]:
# Connect to the Neo4j database
neo4j_driver = GraphDatabase.driver("", auth=("no4j", ""))


In [ ]:
# Load the Excel file
file_path = 'planner.xlsx'

# Read the sheets from the Excel file
calendar_lookup_df = pd.read_excel(file_path, sheet_name='CalendarLookup')
days_lookup_df = pd.read_excel(file_path, sheet_name='DaysLookup')
weeks_lookup_df = pd.read_excel(file_path, sheet_name='WeeksLookup')

# Display the first few rows of each dataframe for review
calendar_lookup_df.head(), days_lookup_df.head(), weeks_lookup_df.head()

# Convert the dataframes to JSON
calendar_lookup_json = calendar_lookup_df.to_json(orient='records')
days_lookup_json = days_lookup_df.to_json(orient='records')
weeks_lookup_json = weeks_lookup_df.to_json(orient='records')

# Combine the JSON objects into a single JSON structure
data = {
    "CalendarLookup": calendar_lookup_json,
    "DaysLookup": days_lookup_json,
    "WeeksLookup": weeks_lookup_json
}

# Parse the individual sheets
calendar_data = json.loads(data['CalendarLookup'])
days_data = json.loads(data['DaysLookup'])
weeks_data = json.loads(data['WeeksLookup'])


In [ ]:
def create_terms_and_breaks(tx, calendar_data):
    node_type = 'time'
    relationship_type = 'time'
    terms_data = []
    term_breaks_data = []
    previous_term_end_date = None

    # Step 1: Create all terms and term breaks
    for item in calendar_data:
        identifier = str(item['Identifier'])

        if 'Term' in identifier and 'Start' in identifier:
            term_number = identifier.replace('Term', '').replace('Start', '')
            start_date_identifier = 'Term' + term_number + 'Start'
            end_date_identifier = 'Term' + term_number + 'End'

            start_date = next((x['Data'] for x in calendar_data if str(x['Identifier']) == start_date_identifier), None)
            end_date = next((x['Data'] for x in calendar_data if str(x['Identifier']) == end_date_identifier), None)

            if start_date and end_date:
                start_date = datetime.datetime.fromtimestamp(start_date / 1000.0)
                end_date = datetime.datetime.fromtimestamp(end_date / 1000.0)

                term_id = tx.run(
                    """
                    MERGE (t:Term {term_number: $term_number, start_date: $start_date, end_date: $end_date, type: $node_type})
                    RETURN id(t) AS term_id
                    """,
                    term_number=term_number,
                    start_date=start_date.date().isoformat(),
                    end_date=end_date.date().isoformat(),
                    node_type=node_type
                ).single()['term_id']

                terms_data.append({
                    'term_id': term_id,
                    'start_date': start_date.timestamp() * 1000,
                    'end_date': end_date.timestamp() * 1000
                })

                # Check for a gap to create TermBreak
                if previous_term_end_date and previous_term_end_date < start_date:
                    term_break_id = tx.run(
                        """
                        CREATE (tb:TermBreak {start_date: $prev_end_date, end_date: $start_date, type: $node_type})
                        RETURN id(tb) AS term_break_id
                        """,
                        prev_end_date=previous_term_end_date.date().isoformat(),
                        start_date=start_date.date().isoformat(),
                        node_type=node_type
                    ).single()['term_break_id']

                    term_breaks_data.append({
                        'term_break_id': term_break_id,
                        'start_date': previous_term_end_date.timestamp() * 1000,
                        'end_date': start_date.timestamp() * 1000
                    })

                previous_term_end_date = end_date

    # Step 2: Establish relationships based on dates
    # Sort nodes by start date
    all_nodes = sorted(terms_data + term_breaks_data, key=lambda x: x['start_date'])

    for i in range(len(all_nodes) - 1):
        current_node = all_nodes[i]
        next_node = all_nodes[i + 1]

        # Create PRECEDES and FOLLOWS relationships
        tx.run(
            """
            MATCH (curr), (next)
            WHERE id(curr) = $current_id AND id(next) = $next_id
            CREATE (curr)-[r1:PRECEDES {type: $relationship_type}]->(next)
            CREATE (next)-[r2:FOLLOWS {type: $relationship_type}]->(curr)
            """,
            current_id=current_node['term_id' if 'term_id' in current_node else 'term_break_id'],
            next_id=next_node['term_id' if 'term_id' in next_node else 'term_break_id'],
            relationship_type=relationship_type
        )

    return terms_data, term_breaks_data


In [ ]:
def find_term_or_break_for_week(week_start_timestamp, terms_data, term_breaks_data):
    print("Finding term or break for week starting:", datetime.datetime.fromtimestamp(week_start_timestamp / 1000.0).date())
    # Convert week_start to a datetime object for comparison
    week_start_date = datetime.datetime.fromtimestamp(week_start_timestamp / 1000.0).date()

    # Check if the week_start is within any of the terms
    for term in terms_data:
        print('Term start date:', datetime.datetime.fromtimestamp(term['start_date'] / 1000.0).date())
        print('Term end date:', datetime.datetime.fromtimestamp(term['end_date'] / 1000.0).date())
        term_start = datetime.datetime.fromtimestamp(term['start_date'] / 1000.0).date()
        term_end = datetime.datetime.fromtimestamp(term['end_date'] / 1000.0).date()
        if term_start <= week_start_date <= term_end:
            print('Week start date falls within term')
            return {'type': 'Term', 'id': term['term_id']}

    # Check if the week_start is within any of the term breaks
    for term_break in term_breaks_data:
        print('Term break start date:', datetime.datetime.fromtimestamp(term_break['start_date'] / 1000.0).date())
        print('Term break end date:', datetime.datetime.fromtimestamp(term_break['end_date'] / 1000.0).date())
        break_start = datetime.datetime.fromtimestamp(term_break['start_date'] / 1000.0).date()
        break_end = datetime.datetime.fromtimestamp(term_break['end_date'] / 1000.0).date()
        if break_start <= week_start_date <= break_end:
            print('Week start date falls within term break')
            return {'type': 'TermBreak', 'id': term_break['term_break_id']}

    # Return None if the week doesn't fall within any term or term break
    print('Week start date falls outside of any term or term break')
    return None


In [ ]:
def create_weeks(tx, weeks_data, terms_data, term_breaks_data):
    node_type = 'time'
    relationship_type = 'time'
    previous_week_id = None
    academic_week = 0

    for week in weeks_data:
        week_type = week['WeekType']
        if week_type in ['A', 'B']:  # Check if the week has type A or B
            academic_week += 1  # Increment academic week only if week_type is A or B

            week_start_timestamp = int(week['WeekStart'])
            week_start_date = datetime.datetime.fromtimestamp(week_start_timestamp / 1000.0).date()
            week_end_date = week_start_date + datetime.timedelta(days=6)  # Add 6 days to get the end date

            term_or_break_id = find_term_or_break_for_week(week_start_timestamp, terms_data, term_breaks_data)

            week_id = tx.run(
                """
                MERGE (w:Week {academic_week: $academic_week, week_start: $week_start, week_end: $week_end, week_type: $week_type, type: $node_type})
                RETURN id(w) AS week_id
                """,
                academic_week=academic_week,
                week_start=week_start_date.isoformat(),
                week_end=week_end_date.isoformat(),
                week_type=week_type,
                node_type=node_type
            ).single()['week_id']

            if previous_week_id is not None:
                tx.run(
                    """
                    MATCH (prev:Week), (curr:Week)
                    WHERE id(prev) = $previous_week_id AND id(curr) = $week_id
                    CREATE (prev)-[r1:FOLLOWS {type: $relationship_type}]->(curr)
                    CREATE (curr)-[r2:PRECEDES {type: $relationship_type}]->(prev)
                    """,
                    previous_week_id=previous_week_id,
                    week_id=week_id,
                    relationship_type=relationship_type
                )

            if term_or_break_id:
                relation_type = "Term" if term_or_break_id['type'] == 'Term' else "TermBreak"
                query = (
                    f"""
                    MATCH (w:Week), (t:{relation_type})
                    WHERE id(w) = $week_id AND id(t) = $term_or_break_id
                    CREATE (w)-[r1:PART_OF_TERM {{type: $relationship_type}}]->(t)
                    CREATE (t)-[r2:CONTAINS_WEEK {{type: $relationship_type}}]->(w)
                    """
                )
                tx.run(query, week_id=week_id, term_or_break_id=term_or_break_id['id'], relationship_type=relationship_type)

            previous_week_id = week_id


In [ ]:
def find_week_for_day(day_date, weeks_data):
    for week in weeks_data:
        week_start = datetime.datetime.fromtimestamp(int(week['WeekStart']) / 1000.0).date()
        week_end = week_start + datetime.timedelta(days=6)  # Assuming a week is 7 days

        if week_start <= day_date <= week_end:
            return week['week_id']  # Use the actual Neo4j node ID of the Week

    # Return None if no matching week is found
    return None


In [ ]:
def find_term_or_break_for_day(day_date, terms_data, term_breaks_data):
    # Check if the day is within any of the terms
    for term in terms_data:
        term_start = datetime.datetime.fromtimestamp(term['start_date'] / 1000.0).date()
        term_end = datetime.datetime.fromtimestamp(term['end_date'] / 1000.0).date()

        if term_start <= day_date <= term_end:
            return {'type': 'Term', 'id': term['term_id']}

    # Check if the day is within any of the term breaks
    for term_break in term_breaks_data:
        break_start = datetime.datetime.fromtimestamp(term_break['start_date'] / 1000.0).date()
        break_end = datetime.datetime.fromtimestamp(term_break['end_date'] / 1000.0).date()

        if break_start <= day_date <= break_end:
            return {'type': 'TermBreak', 'id': term_break['term_break_id']}

    # Return None if the day doesn't fall within any term or term break
    return None


In [ ]:
def create_days(tx, days_data, terms_data, term_breaks_data):
    node_type = 'time'
    relationship_type = 'time'
    previous_day_id = None  # Initialize previous_day_id to None
    academic_day = 0  # Initialize academic_day to 0

    for day in days_data:
        # Convert Unix timestamp to datetime
        day_date = datetime.datetime.fromtimestamp(day['Date'] / 1000.0).date()

        # Query the Neo4j database to find the corresponding week for the current day
        week_result = tx.run(
            """
            MATCH (w:Week) 
            WHERE $day_date >= w.week_start AND $day_date <= w.week_end 
            RETURN id(w) AS week_id, w.week_type AS week_type
            """,
            day_date=day_date.isoformat()
        )

        week_info = week_result.single()
        week_id = week_info['week_id'] if week_info else None
        week_type = week_info['week_type'] if week_info else None

        # Check if day should be created based on DayModifier
        if day['DayModifier'] != 'H':
            # Increment academic_day if day is not a holiday
            academic_day += 1
            # Set day_modifier to week_type if week_type is 'A' or 'B'
            day_modifier = week_type if week_type in ['A', 'B'] else day['DayModifier']

            # Define the properties of the day
            day_properties = {
                'date': day_date.isoformat(),
                'day': day['Day'],
                'day_modifier': day_modifier,
                'academic_day': academic_day,
                'auto_agenda': day['AutoAgenda'] if day['AutoAgenda'] else '',
                'agenda_heading': day['AgendaHeading'] if day['AgendaHeading'] else '',
                'agenda_notes': day['AgendaNotes'] if day['AgendaNotes'] else '',
                'type': node_type
            }

            # Create the current day node
            day_id = tx.run(
                """
                MERGE (d:Day {date: $date})
                ON CREATE SET d.day = $day, d.day_modifier = $day_modifier,
                               d.academic_day = $academic_day, d.auto_agenda = $auto_agenda,
                               d.agenda_heading = $agenda_heading, d.agenda_notes = $agenda_notes,
                                 d.type = $type
                RETURN id(d) AS day_id
                """, **day_properties
            ).single()['day_id']


            # Create precedes and follows relationships between consecutive days
            if previous_day_id is not None:
                tx.run(
                    """
                    MATCH (prev:Day), (curr:Day)
                    WHERE id(prev) = $previous_day_id AND id(curr) = $day_id
                    CREATE (prev)-[r1:FOLLOWS {type: $relationship_type}]->(curr)
                    CREATE (curr)-[r2:PRECEDES {type: $relationship_type}]->(prev)
                    """,
                    previous_day_id=previous_day_id,
                    day_id=day_id,
                    relationship_type=relationship_type
                )

            if week_id is not None:
                print("Creating PART_OF relationship between day and week")
                # Only create the PART_OF relationship if a valid week_id is found
                tx.run(
                    """
                    MATCH (d:Day), (w:Week)
                    WHERE id(d) = $day_id AND id(w) = $week_id
                    MERGE (d)-[r1:PART_OF_WEEK {type: $relationship_type}]->(w)
                    MERGE (w)-[r2:CONTAINS_DAY {type: $relationship_type}]->(d)
                    """,
                    day_id=day_id,
                    week_id=week_id,
                    relationship_type=relationship_type
                )

            # Find the corresponding term or term break for the current day and create PART_OF relationship
            term_or_break_id = find_term_or_break_for_day(day_date, terms_data, term_breaks_data)
            if term_or_break_id:
                relation_type = "Term" if term_or_break_id['type'] == 'Term' else "TermBreak"
                query = (
                    f"""
                    MATCH (d:Day), (t:{relation_type})
                    WHERE id(d) = $day_id AND id(t) = $term_or_break_id
                    MERGE (d)-[r1:PART_OF_TERM {{type: $relationship_type}}]->(t)
                    MERGE (t)-[r2:CONTAINS_DAY {{type: $relationship_type}}]->(d)
                    """
                )
                tx.run(query, day_id=day_id, term_or_break_id=term_or_break_id['id'], relationship_type=relationship_type)

            previous_day_id = day_id


In [ ]:
def create_periods(tx, calendar_data):
    node_type = 'time'
    relationship_type = 'time'
    # Convert calendar_data to a dictionary for easier access
    calendar_dict = {str(item['Identifier']): item['Data'] for item in calendar_data}

    # Extract period information from calendar_data
    period_info = {
        'BS': {'start': '', 'end': calendar_dict['P.Reg.PeriodStartTime']},
        'Reg': {'start': calendar_dict['P.Reg.PeriodStartTime'], 'end': calendar_dict['P.Reg.PeriodEndTime']},
        '1': {'start': calendar_dict['P.1.PeriodStartTime'], 'end': calendar_dict['P.1.PeriodEndTime']},
        '2': {'start': calendar_dict['P.2.PeriodStartTime'], 'end': calendar_dict['P.2.PeriodEndTime']},
        'Brk': {'start': calendar_dict['P.Brk.PeriodStartTime'], 'end': calendar_dict['P.Brk.PeriodEndTime']},
        '3': {'start': calendar_dict['P.3.PeriodStartTime'], 'end': calendar_dict['P.3.PeriodEndTime']},
        '4': {'start': calendar_dict['P.4.PeriodStartTime'], 'end': calendar_dict['P.4.PeriodEndTime']},
        '5': {'start': calendar_dict['P.5.PeriodStartTime'], 'end': calendar_dict['P.5.PeriodEndTime']},
        'Lun': {'start': calendar_dict['P.Lun.PeriodStartTime'], 'end': calendar_dict['P.Lun.PeriodEndTime']},
        '6': {'start': calendar_dict['P.6.PeriodStartTime'], 'end': calendar_dict['P.6.PeriodEndTime']},
        'AS': {'start': calendar_dict['P.6.PeriodEndTime'], 'end': ''}
    }

    # Query each day node in the graph
    days_result = tx.run("MATCH (d:Day) WHERE d.day_modifier IN ['A', 'B'] RETURN id(d) AS day_id, d.date AS date, d.day_modifier AS day_modifier")

    for day_record in days_result:
        day_id = day_record['day_id']
        date = day_record['date']
        week_type = day_record['day_modifier']

        # Query to find the week and term associated with the day
        related_nodes = tx.run(
            """
            MATCH (d:Day)-[:PART_OF_WEEK]->(w:Week), (d)-[:PART_OF_TERM]->(t:Term)
            WHERE id(d) = $day_id
            RETURN id(w) AS week_id, id(t) AS term_id
            """,
            day_id=day_id
        ).single()
        week_id = related_nodes['week_id']
        term_id = related_nodes['term_id']

        previous_period_id = None
        for period_name, times in period_info.items():
            # Combine the date with the period start and end times
            period_start_datetime = f"{date}T{times['start']}" if times['start'] else None
            period_end_datetime = f"{date}T{times['end']}" if times['end'] else None

            # Create period node
            period_id = tx.run(
                """
                CREATE (p:Period {name: $name, start_time: $start, end_time: $end, week_type: $week_type, type: $node_type})
                RETURN id(p) AS period_id
                """,
                name=period_name,
                start=period_start_datetime,
                end=period_end_datetime,
                week_type=week_type,
                node_type=node_type
            ).single()['period_id']

            # Link period to day, week, and term
            tx.run(
                """
                MATCH (d:Day), (w:Week), (t:Term), (p:Period)
                WHERE id(d) = $day_id AND id(w) = $week_id AND id(t) = $term_id AND id(p) = $period_id
                CREATE (p)-[r1:PART_OF_DAY {type: $relationship_type}]->(d), (p)-[r2:PART_OF_WEEK {type: $relationship_type}]->(w), (p)-[r3:PART_OF_TERM {type: $relationship_type}]->(t)
                CREATE (d)-[r4:CONTAINS_PERIOD {type: $relationship_type}]->(p), (w)-[r5:CONTAINS_PERIOD {type: $relationship_type}]->(p), (t)-[r6:CONTAINS_PERIOD {type: $relationship_type}]->(p)
                """,
                day_id=day_id,
                week_id=week_id,
                term_id=term_id,
                period_id=period_id,
                relationship_type=relationship_type
            )

            # Establish sequential relationships
            if previous_period_id:
                tx.run(
                    """
                    MATCH (prev:Period), (curr:Period)
                    WHERE id(prev) = $previous_id AND id(curr) = $period_id
                    CREATE (prev)-[r1:FOLLOWS {type: $relationship_type}]->(curr), (curr)-[r2:PRECEDES {type: $relationship_type}]->(prev)
                    """,
                    previous_id=previous_period_id,
                    period_id=period_id,
                    relationship_type=relationship_type
                )

            previous_period_id = period_id


In [ ]:
# Run the functions to create terms, breaks, weeks, days and periods in the database
with neo4j_driver.session() as session:
    terms_data, term_breaks_data = session.write_transaction(create_terms_and_breaks, calendar_data)

    # Now, call create_weeks with the required data
    session.write_transaction(create_weeks, weeks_data, terms_data, term_breaks_data)

    # Now, call create_days with the required data
    session.write_transaction(create_days, days_data, terms_data, term_breaks_data)

    # Now, call create_periods with the required data
    session.write_transaction(create_periods, calendar_data)

# Close the driver
neo4j_driver.close()


In [ ]:
# Function to parse the 'Period' column and extract the day and time slot
def parse_period(period_str):
    # Split the period string into its components
    week_type = period_str[0]  # 'A' or 'B'
    day_of_week = period_str[1]  # 'M', 'T', 'W', etc.
    period_num = period_str.split(':')[1]  # '1', '2', etc.
    return week_type, day_of_week, period_num

# Function to create timetable entries in Neo4j
def create_timetable_entry(tx, staff_code, teacher, room, week_type, day_of_week, period_num, scheme, subject, class_group):
    tx.run("""
    MERGE (teach:Teacher {name: $teacher, staff_code: $staff_code})
    MERGE (rm:Room {name: $room})
    MERGE (subj:Subject {name: $subject})
    MERGE (cl:Class {name: $class_group})
    MERGE (period:Period {week_type: $week_type, day_of_week: $day_of_week, period_num: $period_num})
    MERGE (lesson:Lesson {week_type: $week_type, day_of_week: $day_of_week, period_num: $period_num, scheme: $scheme, subject: $subject, class_group: $class_group})
    MERGE (teach)-[:TEACHES]->(lesson)
    MERGE (lesson)-[:DURING]->(period)
    MERGE (lesson)-[:IN]->(rm)
    MERGE (subj)-[:SUBJECT_OF]->(lesson)
    MERGE (cl)-[:ATTENDS]->(lesson)
    """, staff_code=staff_code, teacher=teacher, room=room, week_type=week_type, 
         day_of_week=day_of_week, period_num=period_num, scheme=scheme, subject=subject, class_group=class_group)

# Load the Excel file
file_path = 'staff_tt.xlsx'  # Update the path to your file location

# Read the Excel file into a pandas DataFrame
df = pd.read_excel(file_path)


# Close the Neo4j driver
neo4j_driver.close()
